In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

players = pd.read_csv('./Dataset/players.csv')
teams = pd.read_csv('./Dataset/teams.csv')
players_teams = pd.read_csv('./Dataset/players_teams.csv')
awards_players = pd.read_csv('./Dataset/awards_players.csv')
coaches = pd.read_csv('./Dataset/coaches.csv')
series_post = pd.read_csv('./Dataset/series_post.csv')
teams_post = pd.read_csv('./Dataset/teams_post.csv')



In [17]:

players = players.drop('collegeOther', axis=1)

# Have only 0s
players = players.drop('lastseason', axis=1)

players = players[players['height'] > 0]

# Calculate the mean of non-zero heights
mean_height = players[players['height'] != 0]['height'].mean()
# Replace 0 with the mean height
# Replace 9 with the mean height
players['height'] = players['height'].replace(0, mean_height)
players['height'] = players['height'].replace(9, mean_height)

# Calculate the mean of non-zero weights
mean_weight = players[players['weight'] != 0]['weight'].mean()
players['weight'] = players['weight'].replace(0, mean_weight)


In [18]:
def calculate_age(row):
    birth_year = int(row['birthDate'][:4])
    age = 2000 + row['year'] - birth_year
    return age


merged_data = players.merge(players_teams, left_on='bioID', right_on='playerID')

# Apply the function to each row
merged_data['age'] = merged_data.apply(calculate_age, axis=1)

merged_data = merged_data.drop('birthDate', axis=1)
merged_data = merged_data.drop('deathDate', axis=1)



In [19]:


merged_data.to_csv('merged_data.csv', index=False)



In [20]:
# Save the modified DataFrame to a CSV file
players.to_csv('modified_players.csv', index=False)

### Teams

In [21]:

teams = teams.drop(columns=['seeded'], axis=1)
teams = teams.drop(columns=['confID'], axis=1)
teams = teams.drop(columns=['divID'], axis=1)

